In [ ]:
pip install datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.

In [ ]:
import pandas as pd
import numpy as np
from datasets import Dataset
import evaluate
import torch
from transformers import (
    Trainer,
    TrainingArguments,
    AutoTokenizer,
    ModernBertForSequenceClassification
)
label2id = {"negative": 0, "neutral": 1, "positive": 2}
id2label = {0: "negative", 1: "neutral", 2: "positive"}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/Newton/CzechMedia/data/train.csv")
val_df = pd.read_csv("/content/drive/MyDrive/Newton/CzechMedia/data/val.csv")
test_df = pd.read_csv("/content/drive/MyDrive/Newton/CzechMedia/data/test.csv")

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("answerdotai/ModernBERT-base")

def tokenize_function(example):
    question_string = f"Jaký je sentiment tohoto aspektu: {example['aspect']}?"

    encoding = tokenizer(
        text=example["text"],
        text_pair=question_string,
        truncation=True,
        padding="max_length",
        max_length=256
    )
    # Convert label from string to int
    encoding["label"] = label2id[example["label"]]
    return encoding

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

In [ ]:
train_dataset = train_dataset.map(tokenize_function, batched=False)
val_dataset = val_dataset.map(tokenize_function, batched=False)
test_dataset = test_dataset.map(tokenize_function, batched=False)

Map:   0%|          | 0/1896 [00:00<?, ? examples/s]

Map:   0%|          | 0/542 [00:00<?, ? examples/s]

Map:   0%|          | 0/271 [00:00<?, ? examples/s]

In [ ]:
model = ModernBertForSequenceClassification.from_pretrained(
    "answerdotai/ModernBERT-base",
    num_labels=3
)

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/599M [00:00<?, ?B/s]

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=3,
    logging_dir="./logs",
    logging_steps=200,
    warmup_ratio=0.06,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: fialavi2 (fialavi2-czech-technical-university-in-prague) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


W0430 09:39:07.347000 968 torch/_inductor/utils.py:1137] [1/0] Not enough SMs to use max_autotune_gemm mode


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.138015,0.321033
2,1.124100,1.088193,0.433579
3,1.124100,1.038167,0.424354
4,1.032900,1.108256,0.359779
5,1.032900,1.007998,0.466790
6,0.926000,0.807742,0.632841
7,0.557600,0.871918,0.693727
8,0.557600,1.135896,0.712177
9,0.159400,1.462678,0.706642
10,0.159400,1.338920,0.723247


TrainOutput(global_step=1190, training_loss=0.6446042249182693, metrics={'train_runtime': 1542.7269, 'train_samples_per_second': 12.29, 'train_steps_per_second': 0.771, 'total_flos': 3230406775480320.0, 'train_loss': 0.6446042249182693, 'epoch': 10.0})

In [ ]:
val_results = trainer.evaluate()
print(val_results)

{'eval_loss': 1.3389201164245605, 'eval_accuracy': 0.7232472324723247, 'eval_runtime': 10.5821, 'eval_samples_per_second': 51.218, 'eval_steps_per_second': 3.213, 'epoch': 10.0}


In [ ]:
test_results = trainer.evaluate(test_dataset)
print(test_results)

{'eval_loss': 1.4347412586212158, 'eval_accuracy': 0.7527675276752768, 'eval_runtime': 5.3899, 'eval_samples_per_second': 50.279, 'eval_steps_per_second': 3.154, 'epoch': 10.0}


In [ ]:
trainer.save_model("/content/drive/MyDrive/Newton/CzechMedia/models/ModernBERT-baseQA")

In [ ]:
tokenizer.save_pretrained("/content/drive/MyDrive/Newton/CzechMedia/models/ModernBERT-baseQA")

('/content/drive/MyDrive/Newton/CzechMedia/models/ModernBERT-baseQA/tokenizer_config.json',
 '/content/drive/MyDrive/Newton/CzechMedia/models/ModernBERT-baseQA/special_tokens_map.json',
 '/content/drive/MyDrive/Newton/CzechMedia/models/ModernBERT-baseQA/tokenizer.json')

In [ ]:
model = ModernBertForSequenceClassification.from_pretrained("/content/drive/MyDrive/Newton/CzechMedia/models/ModernBERT-baseQA")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Newton/CzechMedia/models/ModernBERT-baseQA")

## Vyhodnocovací metriky

In [ ]:
def compute_metrics_accuracy(eval_pred):
    metric = evaluate.load("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer_accuracy = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics_accuracy
)

In [ ]:
test_results_accuracy = trainer_accuracy.evaluate()
print(test_results_accuracy)

{'eval_loss': 1.4347410202026367, 'eval_model_preparation_time': 0.003, 'eval_accuracy': 0.7527675276752768, 'eval_runtime': 6.0022, 'eval_samples_per_second': 45.15, 'eval_steps_per_second': 5.665}


In [ ]:
def compute_metrics_weightedf1(eval_pred):
    metric = evaluate.load("f1")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average="weighted")

trainer_weightedf1 = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics_weightedf1
)

In [ ]:
test_results_weightedf1 = trainer_weightedf1.evaluate()
print(test_results_weightedf1)

{'eval_loss': 1.4347410202026367, 'eval_model_preparation_time': 0.0035, 'eval_f1': 0.7529954782559216, 'eval_runtime': 6.0605, 'eval_samples_per_second': 44.716, 'eval_steps_per_second': 5.61}


In [ ]:
def compute_metrics_macrof1(eval_pred):
    metric = evaluate.load("f1")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average="macro")

trainer_macrof1 = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics_macrof1
)

In [ ]:
test_results_macrof1 = trainer_macrof1.evaluate()
print(test_results_macrof1)

{'eval_loss': 1.4347410202026367, 'eval_model_preparation_time': 0.0031, 'eval_f1': 0.7526080198732132, 'eval_runtime': 6.0786, 'eval_samples_per_second': 44.583, 'eval_steps_per_second': 5.593}


## Praktické testy

In [ ]:
def predict_sentiment(model, tokenizer, aspect, text):
    question_string = f"Jaký je sentiment tohoto aspektu: {aspect}?"

    encoding = tokenizer(
        text=text,
        text_pair=question_string,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=256
    )

    with torch.no_grad():
        outputs = model(**encoding)

    logits = outputs.logits
    print(logits)
    predicted_class_id = logits.argmax(dim=1).item()
    return id2label[predicted_class_id]

### Sada L

In [ ]:
example_aspect = "výkon"
example_text = "Výkon je dobrý, ale displej je rozmazaný a tmavý. Baterie byla skutečným překvapením, nemusím ji přes den nabíjet!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for L1: {prediction}, expected: positive")
example_aspect = "displej"
example_text = "Výkon je dobrý, ale displej je rozmazaný a tmavý. Baterie byla skutečným překvapením, nemusím ji přes den nabíjet!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for L2: {prediction}, expected: negative")
example_aspect = "baterie"
example_text = "Výkon je dobrý, ale displej je rozmazaný a tmavý. Baterie byla skutečným překvapením, nemusím ji přes den nabíjet!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for L3: {prediction}, expected: positive")

tensor([[-4.9435,  2.6108,  1.5299]])
Predicted sentiment for L1: neutral, expected: positive
tensor([[-7.6895,  2.9881,  4.5870]])
Predicted sentiment for L2: positive, expected: negative
tensor([[-3.9679,  1.0322,  2.1016]])
Predicted sentiment for L3: positive, expected: positive


### Sada R

In [ ]:
example_aspect = "personál"
example_text = "Když jsme vešli do restaurace, personál byl velmi nepříjemný a náš stůl nebyl uklizený. Na druhou stranu bylo jídlo opravdu chutné a pivo bylo vynikající."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for R1: {prediction}, expected: negative")
example_aspect = "stůl"
example_text = "Když jsme vešli do restaurace, personál byl velmi nepříjemný a náš stůl nebyl uklizený. Na druhou stranu bylo jídlo opravdu chutné a pivo bylo vynikající."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for R2: {prediction}, expected: negative")
example_aspect = "jídlo"
example_text = "Když jsme vešli do restaurace, personál byl velmi nepříjemný a náš stůl nebyl uklizený. Na druhou stranu bylo jídlo opravdu chutné a pivo bylo vynikající."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for R3: {prediction}, expected: positive")
example_aspect = "pivo"
example_text = "Když jsme vešli do restaurace, personál byl velmi nepříjemný a náš stůl nebyl uklizený. Na druhou stranu bylo jídlo opravdu chutné a pivo bylo vynikající."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for R4: {prediction}, expected: positive")

tensor([[-4.7702,  1.1990,  1.9399]])
Predicted sentiment for R1: positive, expected: negative
tensor([[-5.2422,  0.2381,  3.3742]])
Predicted sentiment for R2: positive, expected: negative
tensor([[-0.2607, -0.6760, -0.2309]])
Predicted sentiment for R3: positive, expected: positive
tensor([[-0.4389, -2.3563, -0.4199]])
Predicted sentiment for R4: positive, expected: positive


### Sada M

In [ ]:
example_aspect = "Ósace"
example_text = "Na výstavě EXPO 2025 v Ósace představilo mnoho firem své nejnovější produkty. Společnost Saremi úspěšně uvedla nový notebook, který si návštěvníci oblíbili, zatímco nový telefon firmy Babtel byl katastrofa, vůbec nefungoval!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for M1: {prediction}, expected: neutral")
example_aspect = "Saremi"
example_text = "Na výstavě EXPO 2025 v Ósace představilo mnoho firem své nejnovější produkty. Společnost Saremi úspěšně uvedla nový notebook, který si návštěvníci oblíbili, zatímco nový telefon firmy Babtel byl katastrofa, vůbec nefungoval!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for M2: {prediction}, expected: positive")
example_aspect = "Babtel"
example_text = "Na výstavě EXPO 2025 v Ósace představilo mnoho firem své nejnovější produkty. Společnost Saremi úspěšně uvedla nový notebook, který si návštěvníci oblíbili, zatímco nový telefon firmy Babtel byl katastrofa, vůbec nefungoval!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for M3: {prediction}, expected: negative")

tensor([[-6.6112,  2.3097, -2.3490]])
Predicted sentiment for M1: neutral, expected: neutral
tensor([[-6.3842,  1.5672,  6.8759]])
Predicted sentiment for M2: positive, expected: positive
tensor([[-0.4120,  1.1298, -1.7659]])
Predicted sentiment for M3: neutral, expected: negative


### Sada O

In [ ]:
example_aspect = "den"
example_text = "Dneska byl opravdu skvělý den, Anička mě pozvala na rande! Bohužel počasí bylo hrozné, takže jsme to museli zrušit."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for O1: {prediction}, expected: positive")
example_aspect = "počasí"
example_text = "Dneska byl opravdu skvělý den, Anička mě pozvala na rande! Bohužel počasí bylo hrozné, takže jsme to museli zrušit."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for O2: {prediction}, expected: negative")

tensor([[-6.6867, -2.6407,  6.6748]])
Predicted sentiment for O1: positive, expected: positive
tensor([[ 2.2934, -1.6202, -1.3324]])
Predicted sentiment for O2: negative, expected: negative
